<img src='img/logo.png' />

<img src='img/title.png'>

<img src='img/py3k.png'>

# Table of Contents
* [Python's nomenclature for virtual sequences](#Python's-nomenclature-for-virtual-sequences)
* [Generator functions](#Generator-functions)
* [The iterator protocol](#The-iterator-protocol)
* [Generators defining iterables](#Generators-defining-iterables)
	* [Generator comprehensions](#Generator-comprehensions)
* [Exercise (factorization)](#Exercise-%28factorization%29)

# Python's nomenclature for virtual sequences

There are several subtly different terms related to "lazy sequences" in Python.  A *generator
function* is a named function that, when called, returns a *generator*.  In turn, a generator is one particular type of *iterator*.  Other iterators include concrete lists, open file handles, file-like objects like `http.client.HTTPResponse`, views into collections, objects returned by calls to `itertools` functions, etc.

# Generator functions

The simplest generator function possible is:

In [ ]:
def simple():
    yield

What does it do?  Not very much.  The main idea of a generator is that we yield values on demand instead of all at once.  We can yield these values from a value using the `yield` keyword instead of the `return` keyword.  You can think of the `yield` keyword as a "pause" button for the function.  It temporarily suspends execution of the function and yields control to the caller.  The calling function can demand another value from the generator using the `next()` function.

Note that only one `return` statements can ever be executed within a particular function call (but a function might have multiple potential branches that return).  In contrast, we can have multiple `yield` statements inside the function where each one will be executed on subsequent resumptions of the suspended function.

In [ ]:
# A generator function that will yields  values  
def f():
    print("I'm going to yield 0")
    yield 0
    print("I'm going to yield 1")
    yield 1

In [ ]:
# This *only* constructs the generator. Nothing in function is executed.
x = f()  
print("Calling next(x)")
# First next() statement executes up to and including first yield
print(next(x))  
print(next(x))  # Execute up to and including the next yield

What happens when a generator runs out of values? It raises an exception, of course.

Remember a slogan of Python: "Exceptions are not that exceptional." (this might take some getting used to for programmers coming from, e.g. C).

In [ ]:
from traceback import print_exc
try:
    print(next(x))
except Exception:
    print_exc()

# The iterator protocol

There is a protocol for what makes something an *iterator*; and also for what makes it an *iterable* (which are not quite the same thing).  And iterable is simply an object with a `.__iter__()` method, where that method returns an iterator when called.  And iterator is itself an iterable, but one whose `.__iter__()` method generally returns itself.  The extra feature an iterator has over an iterable is that it also requires a `.__next__()` method.

These dunder methods might seem obscure and strange.  But most of their work happens "behind the scenes" and you do not have to think about them (except when you want to).  Basically, these magic methods are a lot like other Python magic methods, and they control how objects respond to basic syntactic constructs.  

Let's illustrate the differences among the types of things:

In [ ]:
from collections.abc import *
def simple():
    yield True
inst = simple()

isinstance(simple, Callable), isinstance(inst, Iterator)

In [ ]:
type(simple), type(inst)

In [ ]:
inst.__next__, inst.__iter__

In [ ]:
l = [1,2,3]
isinstance(l, Iterable), isinstance(l, Iterator)

In [ ]:
type(l), l.__iter__

In [ ]:
try:
    l.__next__
except AttributeError as e:
    print("Lists do not have a .__next__() method")

One powerful use for generators is for representing infinite sequences.  Generators allow us to work with long sequences efficiently.  We can avoid having to calculate and store the sequence all at once in memory.  Below, we represent a common alternating series whose sum converges to $ln(2)$.

In [ ]:
def ln2():
    denom = 1
    sign = 1
    while True:
        yield (1.0/denom)*sign
        denom, sign = denom + 1, sign * -1

In [ ]:
from itertools import islice # very useful for slicing an iterator an iterator
from math import log

In [ ]:
# we slice off the first n terms of the sequence.
sum(islice(ln2(), 100000)) 

In [ ]:
log(2)

We can call other generators.  Lets create a generator that takes care of the just the sign.  It will yield an unending stream of alternating 1, -1, 1, -1, ...

In [ ]:
def altsign(pos=True):
    sign = 1 if pos else -1
    while True:
        yield sign
        sign *= -1
        
def ln2():
    sign = altsign()
    for denom, sign in enumerate(sign, 1):
        yield float(sign)/denom

list(islice(ln2(), 1, 10))

In [ ]:
from itertools import count
altsign2 = ((n%2 * -2)+1 for n in count())
list(islice(altsign(), 1, 10)), list(islice(altsign2, 1, 10))

Generators can also recieve values from the calling method via the ```send()``` method.  This method will send a single object back into a generator.  This object becomes the return value of the ```yield``` statement inside the generator

In [ ]:
def mr_postman():
    letter = None
    while True:
        # Yield, waiting for input
        letter = yield letter
        if not str(letter).isalpha():
            if len(letter) > 1:
                print("Those are not letters")
            else:
                print("That is not a letter")

In [ ]:
f = mr_postman()  # Construct the generator object
next(f)           # Must call next to execute generator to first yield.  
                  # Equivalent to f.send(None)
f.send('g')       # Send a value into the generator.  
                  #   If our postman doesn't receive a string of only 
                  #   letter(s), he will complain
                  # Otherwise he will return the letter(s) he got.

# Generators defining iterables

While you *can* explicitly call `next(it)` or `it.send(val)` repeatedly on the iterators returned by generator functions, the more common pattern by a large margin is to use iterators as sequences (perhaps large or infinite) that you loop through.

In Python, the `StopIteration` exception that we saw is a special signal to loops that a sequence of items is exhausted.  This allows concrete collections like lists to behave the same way as lazy generators for most purposes.

In [ ]:
# A generator function to return letters of a string multiple times
def iterate_letters(s, times=2):
    for letter in s:
        for _ in range(times):
            yield letter
            
for c in iterate_letters("StopIteration", 3):
    print(c, end='_')

Or for another example, remember our `ln2()` generator function defined above.  It successively approximates `math.log(2)` in an iterative way.  We might wonder how long it takes these approximations to get "pretty close" to the true answer (that is, the nearest IEEE-854 floating point number to the true, irrational, answer).

In [ ]:
import math
from itertools import accumulate
delta = .01
log2 = math.log(2)
for i, approx in enumerate(accumulate(ln2())):
    print(i+1, "-", approx)
    if abs(log2-approx) < delta:
        break

## Generator comprehensions

In the Introduction notebooks, we discussed generator comprehensions.  Whether to express a generator as a comprehension or a function is often just a choice of style and readability.  In some sense they are formally equivalent.

In [ ]:
# A simple generator function
def to_upper(s):
    for c in s:
        yield c.upper()

In [ ]:
for c in to_upper("Hello world!"):
    print(c, end='')

In [ ]:
# The same thing as a generator comprehension (but requires name in scope)
s = "Hello world!"
as_upper = (c.upper() for c in s)
for c in as_upper:
    print(c, end='')

In [ ]:
# But we are free to wrap this in a function if we want...
def to_upper2(s):
    return (c.upper() for c in s)

In [ ]:
for c in to_upper2("Hello world!"):
    print(c, end='')

In [ ]:
type(to_upper), type(to_upper2)

In [ ]:
type(to_upper(s)), type(to_upper2(s))

# Exercise (factorization)

Write a generator, that given the number $n$, returns the prime factorization of that number.

Optional: If you have time, write another generator that returns every factorization of the number.

In [ ]:
# For a hint, run this cell
import codecs
print(codecs.encode('''# Hfr fbzr fcrpvny shapgvbaf qrsvarq va nabgure abgrobbx
vzcbeg flf
flf.cngu.nccraq('./fep')
vzcbeg cevzrf''', 'rot13'))

<img src='img/copyright.png'>